In [165]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from ffipy import FFIEC_Client
from io import StringIO


In [166]:
client = FFIEC_Client()
# get a list of RSSD codes for filers on 3/31/2019
filers_RSSD_12312010 = client.retrieve_filers_since_date(ds_name='Call',
                                   reporting_pd_end='12/31/2010',
                                   last_update_date='12/31/2010')
# check how many filers - 5908
len(filers_RSSD_12312010)

6999

In [184]:
failed_banks_df = pd.read_csv('data/failed_banks_dataset.csv')
failed_banks_df['Date'] = pd.to_datetime(failed_banks_df['Closing Date'])
failed_banks_df['Year_failed'] = failed_banks_df['Date'].dt.year
failed_banks_df['Quarter_failed'] = failed_banks_df['Date'].dt.quarter
failed_banks_2010Q4 = failed_banks_df.loc[(failed_banks_df.Year_failed == 2010) & (failed_banks_df.Quarter_failed == 4)]
failed_banks_2010Q4.head()
#failed_banks_2010Q4['CERT'].count()

,Bank Name,City,ST,CERT,Acquiring Institution,Closing Date,Updated Date,Date,Year_failed,Quarter_failed
207,Community National Bank,Lino Lakes,MN,23306,Farmers & Merchants Savings Bank,17-Dec-10,10-Nov-16,2010-12-17,2010,4
208,First Southern Bank,Batesville,AR,58052,Southern Bank,17-Dec-10,29-Jan-19,2010-12-17,2010,4
209,"United Americas Bank, N.A.",Atlanta,GA,35065,State Bank and Trust Company,17-Dec-10,17-Oct-15,2010-12-17,2010,4
210,"Appalachian Community Bank, FSB",McCaysville,GA,58495,Peoples Bank of East Tennessee,17-Dec-10,29-Jan-19,2010-12-17,2010,4
211,Chestatee State Bank,Dawsonville,GA,34578,Bank of the Ozarks,17-Dec-10,21-Sep-15,2010-12-17,2010,4


In [180]:
# Testing Call report for 1 filer
ds_name = 'Call'    # Pull Call report data
end_date = '9/30/2010'  # Pull data in reporting pd ending 3/31/17
fiID_type = 'FDICCertNumber'  # Type of financial inst identifier (ID_RSSD or FDICCertNumber)
fiID = 23306    # Indentifier for Wyomin Bank and Trust
fmt = 'SDF'  # Pull report as PDF
# <facsimileFormat>PDF or XBRL or SDF</facsimileFormat>
outfile = None  # Output file path
return_result = True  # If True, method returns the data
results_bytes2 = client.retrieve_facsimile(ds_name=ds_name, reporting_pd_end=end_date,
                              fiID_type=fiID_type, fiID=fiID,
                              facsimile_fmt=fmt, outfile=None,
                              return_result=True)
report = StringIO(str(results_bytes2, 'utf-8'))
report_df = pd.read_csv(report, sep=';')



In [181]:
report_df.columns = ['Call Date', 'Bank RSSD Identifier', 'ValueID', 'Value', 'Last Update',
       'Short Definition', 'Call Schedule', 'Line Number']

In [205]:
financial_metrics_dict = {'RIAD3210':'Total equity capital',
                          'RCON2170':'Total assets',
                          'RCON3360':'Total loans',
                          'RCONC410':'New nonaccrual assets',
                          'RCONC411': 'Sold nonaccrual assets',
                          'RCON3123':'Allowance for loan losses',
                          'RIAD4093':'Total noninterest expense',
                          'RIAD4340':'Net Income attributable',
                          'RCON2215':'Total transaction deposits',
                          'RCON2385':'Total nontransaction deposits',
                          'RCON3545':'Trading assets'}


In [206]:
fin_columns = list(financial_metrics_dict.keys())
failed_banks_2010Q4 = failed_banks_2010Q4.reindex(columns = np.append(failed_banks_2010Q4.columns.values, fin_columns))

In [207]:
list(fin_columns)

['RIAD3210',
 'RCON2170',
 'RCON3360',
 'RCONC410',
 'RCONC411',
 'RCON3123',
 'RIAD4093',
 'RIAD4340',
 'RCON2215',
 'RCON2385',
 'RCON3545']

In [214]:
selected_metrics = report_df[report_df['ValueID'].isin(fin_columns)]
selected_metrics.drop(columns = ['Call Date', 'Bank RSSD Identifier', 'Last Update',
       'Short Definition', 'Call Schedule', 'Line Number'], axis =1, inplace=True)
selected_metrics.T

,83,89,96,108,119,154,767,768,1362,1385,1405
ValueID,RCON2170,RCON2215,RCON2385,RCON3123,RCON3360,RCON3545,RCONC410,RCONC411,RIAD3210,RIAD4093,RIAD4340
Value,31569,5633,23211,760,16517,0,0,0,2653,2186,-1859


In [209]:
report_df.loc[report_df.ValueID == 'RCON2215']

,Call Date,Bank RSSD Identifier,ValueID,Value,Last Update,Short Definition,Call Schedule,Line Number
89,20100930,367851,RCON2215,5633,20101019,Total,RCE,7


In [188]:
failed_banks_df = failed_banks_2010Q4.

RCON2170 31569
RCON2215 5633
RCON2385 23211
RCON3123 760
RCON3360 16517
RCON3545 0
RCONC410 0
RCONC411 0
RCONG105 2653
RIAD4093 2186
RIAD4340 -1859


In [174]:
len(failed_banks_2010Q4.columns)

32

In [175]:
failed_banks_2010Q4.iloc[0,:]

Bank Name                         Community National Bank
City                                           Lino Lakes
ST                                                     MN
CERT                                                23306
Acquiring Institution    Farmers & Merchants Savings Bank
Closing Date                                    17-Dec-10
Updated Date                                    10-Nov-16
Date                                  2010-12-17 00:00:00
Year_failed                                          2010
Quarter_failed                                          4
RCONG105                                              NaN
RCON2170                                              NaN
RCON3360                                              NaN
RCONC410                                              NaN
RCONC411                                              NaN
RCON3123                                              NaN
RIAD4093                                              NaN
RIAD4340      

In [125]:
test_data = {'2002':['2002', '2003', '2004', None, '2002', '2007'], 
             '2003':['2003', '2003', '2004', None, "bob", '2005'],
             '2004':['2004', None, '2004', '2004', '2007', '2002'],
             '2005':['', '', '', '', '', ''] } 
test_df = pd.DataFrame(test_data) 

test_df.iloc[0, 3] = 1

test_df.iloc[0,:]

2002    2002
2003    2003
2004    2004
2005       1
Name: 0, dtype: object

In [187]:
len(report_df.index)
report_df.head()

,Call Date,Bank RSSD Identifier,ValueID,Value,Last Update,Short Definition,Call Schedule,Line Number
0,20100930,367851,RCON0010,6646,20101019,Cash and balances dues from depository institu...,RCR,34
1,20100930,367851,RCON0020,NaN,20101019,Cash items in process of collection and unpost...,RCA,1a
2,20100930,367851,RCON0071,6074,20101019,Interest-bearing balances,RC,1b
3,20100930,367851,RCON0073,NaN,20101019,Foreign branches of other U.S. banks,RCA,3a
4,20100930,367851,RCON0074,NaN,20101019,Other banks in foreign countries and foreign c...,RCA,3b


In [25]:
RIAD3210 = report_df.loc[report_df.ValueID == 'RIAD3210']['Value'] # Total equity is 2653 for ID=367851

type(RIAD3210)

pandas.core.series.Series

In [24]:
report_df.loc[report_df.ValueID == 'RCON3545']

,Call Date,Bank RSSD Identifier,ValueID,Value,Last Update,Short Definition,Call Schedule,Line Number
154,20100930,367851,RCON3545,0,20101019,Trading Assets,RCR,41


In [176]:
report_df.loc[report_df.ValueID == 'RCONG105'] # Total equity is 2653 for ID=367851, SAME AS ABOVE!!!

,Call Date,Bank RSSD Identifier,ValueID,Value,Last Update,Short Definition,Call Schedule,Line Number


In [177]:
report_df.loc[report_df.ValueID == 'RCON2170']['Value'] # Total assets is  31569 for ID=367851

0       6646
1        NaN
2       6074
3        NaN
4        NaN
5        NaN
6        572
7        NaN
8        NaN
9        NaN
10         0
11         0
12      2809
13         0
14         0
15         0
16         0
17         0
18       NaN
19       NaN
20       NaN
21       NaN
22       NaN
23       NaN
24         0
25         0
26         0
27         0
28         0
29         0
        ... 
1657     NaN
1658     NaN
1659     NaN
1660     NaN
1661     NaN
1662     NaN
1663     NaN
1664     NaN
1665     NaN
1666     NaN
1667     NaN
1668     NaN
1669     NaN
1670     NaN
1671     NaN
1672     NaN
1673     NaN
1674     NaN
1675     NaN
1676     NaN
1677     NaN
1678     NaN
1679     NaN
1680     NaN
1681     NaN
1682     NaN
1683     NaN
1684     NaN
1685     NaN
1686     NaN
Name: Value, Length: 1687, dtype: object

In [41]:
EQTA = RIAD3210 / RCON2170 # 1) Total Equity / Total Assets (EQTA) = 0.08 for ID=367851

In [52]:
#RCON3360 = report_df.loc[report_df.ValueID == 'RCON3360'] #Total loans of 16517  for ID=367851
#EQTL = RCONG105 / RCON3360 # Total Equity / Total Loans (EQTL)

In [11]:
#NPLTA = Non Performing Loans / Total Assets (NPLTA)
#RCONC410  #Additions to nonaccrual assets during the previous six months
#RCONC411 # Nonaccrual assets sold during the previous six months

#LLRTA = Loan Loss Reserves / Total Assest
# RCON3123     # Allowance for loan and lease losses

#LLRGL = Loan Loss Reserves / Gross Loans (LLRGL)


# (M)anagement quality
#       7) Operating Expenses / Assets (OEXTA)
# RIAD4093 # Total noninterest expense (sum of items 7.a through 7.d).




#       - Operating Costs / Net Operating Income (OCNOI)
#
#
# (E)arnings ability
#       8) Net Income / Total Assest (ROA)
# RIAD4340    # Net Income (Net income (loss) attributable to bank (item 12 minus item 13).)
#       9) Net Income / Total Equity (ROE)
# 
#
#
#(L)iquidity, i.e. ability to fulfill its short-term liability
#       10) Total Loans / Total Customer Deposits (TLTD)
#  RCON2215  # Total transaction accounts deposits 
#  RCON2385  # Total non-transaction accounts 
#
#
#       11) Total Customer Deposits / Total Assest (TDTA)
#
#
#
# (S)ensitivity to market risk 
#  RCON3545 # Trading Assets /Total assest
#
#
#  Schedule RC-B: Securities 
#  RCON1754 # (Column A) Held-to-maturity Amortized Cost
#  RCON1771 # (Column B) Held-to-maturity Fair Value
#  RCON1772 # (Column C) Available-for-sale Amortized Cost
#  RCON1773 # (Column D) Available-for-sale Fair Value


In [10]:
report_df.loc[report_df.ValueID == 'RCONB549'] # Trading Assets

,Call Date,Bank RSSD Identifier,ValueID,Value,Last Update,Short Definition,Call Schedule,Line Number
443,20100930,367851,RCONB549,5469,20101019,"Individuals, partnerships, and corporations (i...",RCE,1
